In [1]:
import torch

In [2]:
def _get_feat_extract_output_lengths(conv_feature_layers, input_lengths: torch.LongTensor):
        """
        Computes the output length of the convolutional layers
        """

        def _conv_out_length(input_length, kernel_size, stride):
            return torch.floor((input_length - kernel_size) / stride + 1)

        conv_cfg_list = conv_feature_layers

        for i in range(len(conv_cfg_list)):
            input_lengths = _conv_out_length(
                input_lengths, conv_cfg_list[i][1], conv_cfg_list[i][2]
            )

        return input_lengths.to(torch.long)

In [3]:
padding_mask = torch.tensor([
    [False, False, False, False, False, False, False, False, False, False, False, False],
    [False, False, False, False, False, False, False, False, True, True,True, True]
])
padding_mask

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False],
        [False, False, False, False, False, False, False, False,  True,  True,
          True,  True]])

In [4]:
input_lengths= (1 - padding_mask.long()).sum(-1)
conv_feature_layers = [[512, 3, 1], [512, 2, 1], [512, 2, 1], [512, 2, 1], [512, 2, 1]]
output_lengths=_get_feat_extract_output_lengths(conv_feature_layers, input_lengths)

In [5]:
padding_mask = torch.zeros(2, 10)
padding_mask

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [6]:
padding_mask[
                (
                    torch.arange(padding_mask.shape[0]),
                    output_lengths - 1,
                )
            ] = 1

In [7]:
output_lengths

tensor([6, 2])

In [8]:
padding_mask

tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [9]:
padding_mask = (1 - padding_mask.flip([-1]).cumsum(-1).flip([-1])).bool()
padding_mask

tensor([[False, False, False, False, False, False,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True,  True,  True,  True,  True]])